In [33]:
import numpy as np
import os
import re
from scipy.io import loadmat
import open3d
import scipy
from scipy.spatial import distance
from open3d import geometry
from tqdm import tqdm
def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1, normals = []):
    
    valid_int = (a_1[:,2])<1
    a_1 = a_1[valid_int]
    if not len(normals) == 0:
        normals = normals[valid_int]
        return a_1, normals
    
    return a_1

def remove_nan(points, normals):
    keep = []
    for nan in np.isnan(normals):
        if (nan == True).any():
            keep.append(False)
        else:
            keep.append(True)
    points = points[keep]
    normals = normals[keep]
    return (points, normals)

def rigid_motion(p,q):
    """
    Least-Squares Rigid Motion Using Singular Value Decomposition. 
    (https://igl.ethz.ch/projects/ARAP/svd_rot.pdf) 
    
    (note: so far only for the easy case, where all weights are = 1)
    
    p,q: shape [num_points, 3]
    
    """
    n,d = p.shape
    
    # compute centroids
    p_cen = sum(p)/len(p)
    q_cen = sum(q)/len(q)
    
    # compute centered vectors
    X = np.array([i-p_cen for i in p])
    Y = np.array([i-q_cen for i in q])
    
    # compute covariance matrix 
    W = np.eye(n)
    S =  X.T.dot(W).dot(Y)
    
    # compute sigular value decomposition
    U, _, V = np.linalg.svd(S)
    
    # compute optimal R and t
    M = np.eye(d)
    M[-1,-1] = np.linalg.det(V.T.dot(U.T))
    R = V.T.dot(M).dot(U.T)
    
    t = q_cen - R.dot(p_cen)
    
    return R, t

def rms_error(p, q):
    n = p.shape[0]
    dist = [distance.euclidean(p[i,:], q[i,:]) for i in range(n)]
    return np.sqrt(np.sum(np.power(dist, 2))/n)

def show_fitting_result(pcds_list):
    
    point_clouds_object_list = []
    pc = open3d.PointCloud()
    for i, pcd in enumerate(pcds_list):
        point_clouds_object_list.append(open3d.PointCloud())
        point_clouds_object_list[i].points = open3d.Vector3dVector(pcd)
    
    open3d.draw_geometries(point_clouds_object_list)

    
def informative_subsampling(normals, sample_size):
    # convert normals to angular space
    b = np.sqrt(normals[:,0]**2+normals[:,1]**2)
    x = np.arctan2(normals[:,1], normals[:,0])
    y = np.arctan2(normals[:,2], b)
    
    # devide normals over bins
    bins = np.linspace(-np.pi, np.pi, sample_size) 
    x_index = np.digitize(x, bins, right=True)
    y_index = np.digitize(y, bins, right=True)
    index = x_index * sample_size + y_index

    # uniformly sample from bins
    unique_index, original_index = np.unique(index, return_index=True)
    samples = np.random.choice(unique_index.shape[0], sample_size, replace=False)
    sample_index = original_index[samples]
    
    # return only the found sample indices of the original pointcloud
    return sample_index    
    
    
    
    
def icp(a_1, a_2, convergence_treshold=0.0005, point_selection="all", sample_size=1000, generate_3d = True, verbose = True, is_test = False , accuracy_check = False, stability_constant = 1,source_file=None , target_file=None, normals = [], no_background_removal = False):
    """
    a_1: positions of points in point cloud 1. shape : [num_points1, 3]
    a_2: positions of points in point cloud 2. shape : [num_points2, 3]
    
    """
    
    if is_test:
        generate_3d = False
        verbose = False
    n,d = a_1.shape
    
    if not no_background_removal:
    
    # Filter the point clouds based on the depth,
    # only keep the indices where the z of the point cloud is less than 1
        if len(normals) == 0:
            a_1, a_2 = background_removal(a_1), background_removal(a_2)
        else:
            (a_1, normals[0]) = background_removal(a_1, normals[0])
            (a_2, normals[1]) = background_removal(a_2, normals[1])
    
    a_2_c = a_2.copy()
    # Point selection
    # Uniform subsampling
    if point_selection == "uniform":
        a_1 = a_1[np.random.randint(low=0, high=a_1.shape[0], size=sample_size)]
        a_2 = a_2[np.random.randint(low=0, high=a_2.shape[0], size=sample_size)]
        
    if point_selection == "informative":
        a_1 = a_1[informative_subsampling(normals[0], sample_size)]
        a_2 = a_2[informative_subsampling(normals[1], sample_size)]
                 
    if stability_constant == 1 :
        R_overall = np.eye(d)
        t_overall = np.zeros(d)
    else:
        R_overall = np.random.normal(0,1, size = (d,d))*stability_constant
        t_overall = np.random.normal(0,1, size = d)*stability_constant
    
    # Base loop on difference in rsm error
    rms_error_old = 10000
    rms_error_new = rms_error_old-1
    
    while rms_error_old-rms_error_new > convergence_treshold:
        
        if point_selection == "random":
            a_1 = a_1[np.random.choice(a_1.shape[0], sample_size, replace=False), :]
            a_2 = a_2[np.random.choice(a_2.shape[0], sample_size, replace=False), :]
            
        # (Step 1) Find closest points for each point in a_1 from a_2
        tree = scipy.spatial.KDTree(a_2)
        closest_dists, closest_idx = tree.query(a_1)
        # Found this on stackoverflow: https://bit.ly/2P8IYiw
        # Not sure if we can use this, but it is definetely much (!!) faster 
        # than manually comparing all the vectors.
        # Usage also proposed on Wikipedia: https://bit.ly/2urg9nU
        # For how-to-use see: https://bit.ly/2UbKNfn
        closest_a_2 = a_2[closest_idx]
    
        # (Step 2) Refine R and t using Singular Value Decomposition
        R, t = rigid_motion(a_1,closest_a_2)
       
        # update a_1
        a_1 =affine_transform(a_1, R, t)
        
        # update rms error
        rms_error_old = rms_error_new
        rms_error_new = rms_error(a_1, closest_a_2)
        
        if verbose:
            print(rms_error_new)
        
        # update overall R and t
        R_overall = R.dot(R_overall)
        t_overall = R.dot(t_overall) + t
    if generate_3d:
        show_fitting_result([a_1, a_2_c])
    if accuracy_check:
        return rms_error_new
    return R_overall, t_overall
    

In [59]:
# print("Load a ply point cloud, print it, and render it")
# pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
# pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
# print(np.asarray(pcd.points))
# open3d.draw_geometries([pcd])
a_1 = loadmat("Data/source.mat")["source"].T
a_2 = loadmat("Data/target.mat")["target"].T

a_1 = open3d.read_point_cloud("Data/data/0000000000.pcd")
a_2 = open3d.read_point_cloud("Data/data/0000000010.pcd")

a_1 = np.asarray(a_1.points)
a_2 = np.asarray(a_2.points)

n_1 = open3d.read_point_cloud("Data/data/0000000000_normal.pcd", format='xyz')
n_2 = open3d.read_point_cloud("Data/data/0000000010_normal.pcd", format='xyz')

n_1 = np.asarray(n_1.points)
n_2 = np.asarray(n_2.points)


a_1, n_1  = remove_nan(a_1, n_1)
a_2, n_2 = remove_nan(a_2, n_2)


#source_file = "Data/data/0000000000.jpg"
#target_file = "Data/data/0000000001.jpg"

In [62]:
icp(a_1, a_2, point_selection = 'informative', normals = [n_1, n_2] )

0.03258754321643644
0.022913955786016616
0.02115971201033881
0.020248994232444494
0.019792583746345353


(array([[ 0.87997192, -0.03765211, -0.47353115],
        [ 0.05502664,  0.9982226 ,  0.02288491],
        [ 0.47182783, -0.0461949 ,  0.88047972]]),
 array([ 0.43097789, -0.02823118,  0.09698219]))

In [2]:
sampling_methods = ['random', 'uniform', "all"] 
np.random.seed(42)


In [43]:
def speed_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        print("Time that it takes for {} method is:".format(method))
        %time  icp(a_1, a_2, point_selection = method, is_test = True , normals=normals)
speed_check(a_1, a_2, sampling_methods)

Time that it takes for random method is:
CPU times: user 8.13 s, sys: 203 ms, total: 8.34 s
Wall time: 5.12 s
Time that it takes for uniform method is:
CPU times: user 7.9 s, sys: 169 ms, total: 8.06 s
Wall time: 5.59 s
Time that it takes for all method is:
CPU times: user 47.2 s, sys: 4.01 s, total: 51.3 s
Wall time: 43.3 s


In [46]:
def accuracy_check(a_1, a_2, sampling_methods):
    for method in sampling_methods:
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        print("RMSE of {} method is {:.4}".format(method, icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, normals = normals)))
accuracy_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.08269
RMSE of uniform method is 0.08705
RMSE of all method is 0.02899


In [47]:
def  noise_check(a_1, a_2, sampling_methods):
    noise_1, noise_2 = np.random.normal(0,1,(a_1.shape)) ,np.random.normal(0,1,(a_2.shape))
    a_1_noisy, a_2_noisy = a_1 + noise_1 , a_2 + noise_2
    
    for method in sampling_methods:
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, normals = normals) 
        
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        
        
        
        rmse_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True, accuracy_check = True, normals = normals)
        print("RMSE of {} method is {:.4}, whereas if we add noise, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_noisy, rmse_normal - rmse_noisy))
       
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
            
            
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True, normals = normals) 
        
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        
        R_noisy , t_noisy = icp(a_1_noisy, a_2_noisy, point_selection = method, is_test = True, normals = normals) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_noisy), np.linalg.norm(t_normal - t_noisy)
        print("The distance between normal and noisy R matrices is {:.4} and between normal and noisy t vectors is {:.4}".format(R_distance, t_distance))
        

noise_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.07941, whereas if we add noise, it becomes 0.3269 , the difference is -0.2475
The distance between normal and noisy R matrices is 0.5351 and between normal and noisy t vectors is 0.3728
RMSE of uniform method is 0.08523, whereas if we add noise, it becomes 0.3513 , the difference is -0.2661
The distance between normal and noisy R matrices is 0.4378 and between normal and noisy t vectors is 0.3449
RMSE of all method is 0.02899, whereas if we add noise, it becomes 0.2066 , the difference is -0.1776
The distance between normal and noisy R matrices is 0.4928 and between normal and noisy t vectors is 0.5501


In [48]:
def  stability_check(a_1, a_2, sampling_methods):
    stability_constant = 10
    for method in sampling_methods:
        
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        rmse_normal =icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, normals = normals) 
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        
        rmse_non_stable= icp(a_1, a_2, point_selection = method, is_test = True, accuracy_check = True, stability_constant = stability_constant, normals = normals)
        print("RMSE of {} method is {:.4}, whereas if we have random initialisation, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_non_stable, rmse_normal - rmse_non_stable))
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        R_normal , t_normal = icp(a_1, a_2, point_selection = method, is_test = True, normals = normals) 
        if method == "informative":
            normals = [n_1, n_2]
        else:
            normals = []
        R_non_stable , t_non_stable = icp(a_1, a_2, point_selection = method, is_test = True, stability_constant = stability_constant, normals = normals) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_non_stable), np.linalg.norm(t_normal - t_non_stable)
        print("The distance between normal and randomely initialised R matrices is {:.4} and between normal and randomely initialised t vectors is {:.4}".format(R_distance, t_distance))
        

stability_check(a_1, a_2, sampling_methods)
    

RMSE of random method is 0.07902, whereas if we have random initialisation, it becomes 0.08193 , the difference is -0.002905
The distance between normal and randomely initialised R matrices is 32.82 and between normal and randomely initialised t vectors is 23.8
RMSE of uniform method is 0.08893, whereas if we have random initialisation, it becomes 0.08164 , the difference is 0.007296
The distance between normal and randomely initialised R matrices is 32.82 and between normal and randomely initialised t vectors is 20.72
RMSE of all method is 0.02899, whereas if we have random initialisation, it becomes 0.02899 , the difference is 0.0
The distance between normal and randomely initialised R matrices is 22.01 and between normal and randomely initialised t vectors is 11.28


Note: Take stability as an example, an experimental setup may be the following: given a source point cloud, augmented target point cloud can be obtained by transforming the source point cloud using a random rigid transform R and t. Stability can be analyzed by observing a behaviour of ICP on source and augmented target point cloud w.r.t changes in magnitude of R and t.
        

- stability is about convergence of the algorithm dependent on initial condition.

- tolerance to noise is about convergence of the algorithm with input data with noise. You can imagine data is captured by a sensor. In the ideal case you will obtain exact point cloud, however sensor is not precise, therefore there will be noise in measurement. Therefore we ask you to evaluate how ICP is robust against those kind of issuses.

In [24]:

#data preparation cell
def read_pcds_from_filenames(filenames, is_normal = False):
    """
    Read point clouds for given file names.
    """
    if not is_normal:
        return [open3d.read_point_cloud(f)for f in filenames]
    else: 
        return [open3d.read_point_cloud(f, format = "xyz")for f in filenames]

def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1):
    valid_bool_1 = (a_1[:,2])<1
    a_1 = a_1[valid_bool_1]
    return a_1

def prepare_data(frame_interval=1, data_dir="./Data/data/", max_images = 99):
    filenames = []#data_dir+x for x in os.listdir(data_dir) if re.match(r"00000000[0-9][0-9].pcd",x)]
    normals_filenames = []#data_dir+x for x in #s.listdir(data_dir) if re.match(r"00000000[0-9][0-9]_normal.pcd",x)]
    for i in range(max_images+1):
        if i<10:
            filenames.append("{}000000000{}.pcd".format(data_dir, i))
            normals_filenames.append("{}000000000{}_normal.pcd".format(data_dir, i))
        else:
            filenames.append("{}00000000{}.pcd".format(data_dir, i))
            normals_filenames.append("{}00000000{}_normal.pcd".format(data_dir, i))
    

    normals_filenames.sort()
    filenames.sort()
    
    # Get relevant filenames (according to frame_interval)
    filenames=filenames[0::frame_interval]
    normals_filenames = normals_filenames[0::frame_interval]
    
    # Get point clouds for relevant filnames
    pcds = read_pcds_from_filenames(filenames)
    normals = read_pcds_from_filenames(normals_filenames, True)
    normals = [np.array(n.points) for n in normals]

    
    pcd_points = [background_removal(remove_nan(np.array(p.points), n)[0]) for p, n in zip(pcds, normals)]
    
    return pcd_points



pcd_points = prepare_data(frame_interval=1)


In [23]:
#3.1 approach 1

def merging_scenes(pcd_points= pcd_points, point_selection='uniform'):
    
  
    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    transformed_points = []
    
    for i in tqdm(range(1, len(pcd_points))):
        
        # Perform ICP
        R,t = icp(pcd_points[i], pcd_points[i-1], is_test=True, point_selection=point_selection)
        
        # Update transformations back to zero frame
        R_to_zero_current = R.dot(R_to_zero_current)
        t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        
        transformed_points += [affine_transform(pcd_points[i][np.random.randint(low=0, high=pcd_points[i].shape[0], size=600)] , R_to_zero_current , t_to_zero_current)]
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        
    show_fitting_result(transformed_points)
    
    
merging_scenes()




  0%|          | 0/99 [00:00<?, ?it/s]




ValueError: shapes (3,3) and (100,3) not aligned: 3 (dim 1) != 100 (dim 0)

In [32]:
#3.1 approach 2
def affine_transform(pc, R, t):
    return np.array([R.dot(x)+t  for x in pc])

def merging_scenes(pcd_points, point_selection='uniform'):


    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    
    

    total =  pcd_points[0]#[np.random.randint(low=0, high=pcd_points[0].shape[0], size=1000)]
    for i in tqdm(range(0, len(pcd_points)-1)):
        
        # Perform ICP
        R, t = icp(pcd_points[i], pcd_points[i+1], is_test=True, point_selection=point_selection)
        
        # Update transformations back to zero frame
        #R_to_zero_current = R.dot(R_to_zero_current)
        #t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        total = affine_transform(total, R, t)
        total = np.concatenate((total,pcd_points[i+1][np.random.randint(low=0, high=pcd_points[i+1].shape[0], size=600)] )) 
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        
    show_fitting_result(total)
    
    
merging_scenes(pcd_points)





  0%|          | 0/99 [00:00<?, ?it/s]



  1%|          | 1/99 [00:02<03:33,  2.18s/it]





KeyboardInterrupt: 

In [26]:

#3.2
def affine_transform_new(pc, R, t):
    return [R.dot(x)+t for x in pc ]

def merging_scenes(frame_interval=1, point_selection='uniform', data_dir="./Data/data/", max_images =99):
    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    
    
    
    transformed_points =  pcd_points[0][np.random.randint(low=0, high=pcd_points[0].shape[0], size=1010)]
    for i in tqdm(range(1, len(pcd_points)-1)):
        
        # Perform ICP=]
        
        sample_next_frame = pcd_points[i]#[np.random.randint(low=0, high=pcd_points[i].shape[0], size=1000),:]
        #[np.random.randint(low=0, high=transformed_points.shape[0], size=transformed_points.shape[0]),:]
        R,t = icp( transformed_points, sample_next_frame, is_test=True, point_selection=point_selection, no_background_removal = True)
        
        # Update transformations back to zero frame
        #R_to_zero_current = R.dot(R_to_zero_current)
        #t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        transformed_points = affine_transform(transformed_points, R, t)
        
        
        transformed_points = np.concatenate((transformed_points, sample_next_frame[np.random.randint(low=0, high=sample_next_frame.shape[0], size=14000),:]))
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        if i%20==0:
            show_fitting_result([transformed_points])
    show_fitting_result([transformed_points])
    
merging_scenes()




  0%|          | 0/98 [00:00<?, ?it/s]


  1%|          | 1/98 [00:00<00:24,  4.01it/s]


  2%|▏         | 2/98 [00:00<00:28,  3.40it/s]


  3%|▎         | 3/98 [00:01<00:37,  2.56it/s]


  4%|▍         | 4/98 [00:02<00:50,  1.88it/s]


  5%|▌         | 5/98 [00:03<00:58,  1.60it/s]


  6%|▌         | 6/98 [00:04<01:08,  1.33it/s]


  7%|▋         | 7/98 [00:05<01:15,  1.20it/s]


  8%|▊         | 8/98 [00:08<01:37,  1.08s/it]


  9%|▉         | 9/98 [00:11<01:52,  1.26s/it]


 10%|█         | 10/98 [00:14<02:03,  1.40s/it]


 11%|█         | 11/98 [00:16<02:10,  1.50s/it]


 12%|█▏        | 12/98 [00:19<02:19,  1.62s/it]


 13%|█▎        | 13/98 [00:22<02:25,  1.71s/it]

KeyboardInterrupt: 

In [31]:
#3.2 approach 2

def merging_scenes(pcd_points= pcd_points, point_selection='uniform'):
    
  
    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    transformed_points =  pcd_points[0][np.random.randint(low=0, high=pcd_points[0].shape[0], size=1010)]

    for i in tqdm(range(1, len(pcd_points))):
        
        # Perform ICP
        R,t = icp(pcd_points[i], transformed_points, is_test=True, point_selection=point_selection)
        
        # Update transformations back to zero frame
        #R_to_zero_current = R.dot(R_to_zero_current)
        #t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        sample_transformed = affine_transform(pcd_points[i][np.random.randint(low=0, high=pcd_points[i].shape[0], size=600)], R, t)
        transformed_points = np.concatenate((transformed_points,sample_transformed))#[affine_transform(pcd_points[i][np.random.randint(low=0, high=pcd_points[i].shape[0], size=600)] , R_to_zero_current , t_to_zero_current)]
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        
    show_fitting_result([transformed_points])
    
    
merging_scenes()





  0%|          | 0/99 [00:00<?, ?it/s]



  1%|          | 1/99 [00:00<00:25,  3.91it/s]



  2%|▏         | 2/99 [00:00<00:22,  4.34it/s]



  3%|▎         | 3/99 [00:00<00:22,  4.22it/s]



  4%|▍         | 4/99 [00:00<00:22,  4.14it/s]



  5%|▌         | 5/99 [00:01<00:24,  3.88it/s]



  6%|▌         | 6/99 [00:01<00:25,  3.71it/s]



  7%|▋         | 7/99 [00:02<00:27,  3.34it/s]



  8%|▊         | 8/99 [00:02<00:28,  3.22it/s]



  9%|▉         | 9/99 [00:02<00:28,  3.17it/s]



 10%|█         | 10/99 [00:03<00:27,  3.19it/s]



 11%|█         | 11/99 [00:03<00:27,  3.23it/s]



 12%|█▏        | 12/99 [00:03<00:27,  3.21it/s]



 13%|█▎        | 13/99 [00:03<00:26,  3.29it/s]



 14%|█▍        | 14/99 [00:04<00:25,  3.30it/s]



 15%|█▌        | 15/99 [00:04<00:25,  3.31it/s]



 16%|█▌        | 16/99 [00:04<00:25,  3.28it/s]



 17%|█▋        | 17/99 [00:05<00:24,  3.29it/s]



 18%|█▊        | 18/99 [00:05<00:25,  3.21it/s]



 19%|█▉        | 19/99 [00:05<00:24,  3.22it/